In [107]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

- Завантаження даних "https://drive.google.com/file/d/1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo/view"

In [108]:
url = "https://drive.google.com/uc?export=download&id=1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo"
df = pd.read_csv(url)

In [109]:
df.sample(10)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
99,6265000,6000,4,1,3,yes,yes,yes,no,no,0,yes,unfurnished
486,2870000,6000,2,1,1,yes,no,no,no,no,0,no,semi-furnished
5,10850000,7500,3,3,1,yes,no,yes,no,yes,2,yes,semi-furnished
374,3640000,3520,2,2,1,yes,no,yes,no,no,0,no,semi-furnished
80,6629000,6000,3,1,2,yes,no,no,yes,no,1,yes,semi-furnished
432,3290000,6060,3,1,1,yes,yes,yes,no,no,0,no,furnished
236,4620000,2870,2,1,2,yes,yes,yes,no,no,0,yes,semi-furnished
73,6685000,6600,2,2,4,yes,no,yes,no,no,0,yes,furnished
79,6650000,6000,3,2,3,yes,yes,no,no,yes,0,no,furnished
451,3150000,6750,2,1,1,yes,no,no,no,no,0,no,semi-furnished


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [111]:
df[df.isna()].count()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [112]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


- Підготовка даних

In [113]:
X = df[['area', 'bathrooms', 'bedrooms']].values
y = df['price'].values

- Нормалізація даних

In [114]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled 

array([[ 1.04672629,  1.42181174,  1.40341936],
       [ 1.75700953,  5.40580863,  1.40341936],
       [ 2.21823241,  1.42181174,  0.04727831],
       ...,
       [-0.70592066, -0.57018671, -1.30886273],
       [-1.03338891, -0.57018671,  0.04727831],
       [-0.5998394 , -0.57018671,  0.04727831]])

- Додавання колонки з одиницями для врахування зміщення

In [115]:
X_scaled = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]

# 1. напишіть функцію гіпотези лінійної регресії у векторному вигляді


X: матриця ознак (з доданим стовпцем одиниць)

w: вектор параметрів

return: вектор передбачень

In [116]:
def hypothesis(X, w):
    return np.dot(X, w)

# 2. створіть функцію для обчислення функції втрат у векторному вигляді 
## (MSE)

X: матриця ознак (з доданим стовпцем одиниць)

y: вектор цільових значень

w: вектор параметрів

return: значення функції втрат

In [117]:
def mse(X, y, w):
    m = len(y)
    return np.mean((hypothesis(X, w) - y) ** 2)   # return np.sum((hypothesis(X, w) - y) ** 2) / (2 * m)

# 3. реалізуйте один крок градієнтного спуску
    
X: матриця ознак (з доданим стовпцем одиниць)

y: вектор цільових значень

w: поточний вектор параметрів

learning_rate: швидкість навчання

return: оновлений вектор параметрів

In [118]:
def gradient_step(X, y, w, learning_rate):
    m = len(y)
    gradient = np.dot(X.T, (hypothesis(X, w) - y)) / m
    return w - learning_rate * gradient

# 4. знайдіть найкращі параметри w для датасету 
## використовуючи написані вами функції, прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень

## знаходження оптимальних параметрів за допомогою градієнтного спуску
    
X: матриця ознак (з доданим стовпцем одиниць)

y: вектор цільових значень

learning_rate: швидкість навчання

num_iterations: кількість ітерацій

return: оптимальний вектор параметрів


In [124]:
def gradient_descent(X, y, learning_rate=0.01, num_iterations=1000):

    w = np.zeros(X.shape[1])
    for _ in range(num_iterations):
        w = gradient_step(X, y, w, learning_rate)
    return w

In [125]:
w_gd = gradient_descent(X_scaled, y)
w_gd

array([4766523.46205873,  821199.26709864,  695515.99623791,
        300296.28560637])

5. знайдіть ці ж параметри за допомогою аналітичного рішення;

In [121]:
w_analytical = np.linalg.inv(X_scaled.T.dot(X_scaled)).dot(X_scaled.T).dot(y)
w_analytical

array([4766729.24770642,  821214.14349519,  695808.52272537,
        299983.57107963])


6. для перевірки спрогнозованих значень, використайте LinearRegression з бібліотеки scikit-learn та порівняйте результати. 

In [122]:
lr = LinearRegression()
lr.fit(X_scaled[:, 1:], y)  # не включаємо стовпець з одиницями
w_sklearn = np.concatenate(([lr.intercept_], lr.coef_))
w_sklearn

array([4766729.24770642,  821214.14349519,  695808.52272537,
        299983.57107963])

# Порівняння результатів

In [123]:
y_pred_gd = hypothesis(X_scaled, w_gd)
y_pred_analytical = hypothesis(X_scaled, w_analytical)
y_pred_sklearn = lr.predict(X_scaled[:, 1:])


mse_gd = np.mean((y - y_pred_gd) ** 2)
mse_analytical = np.mean((y - y_pred_analytical) ** 2)
mse_sklearn = np.mean((y - y_pred_sklearn) ** 2)

print("Середньоквадратична помилка:")
print("Градієнтний спуск:", mse_gd)
print("Аналітичне рішення:", mse_analytical)
print("Scikit-learn:", mse_sklearn)

Середньоквадратична помилка:
Градієнтний спуск: 1791170207770.2297
Аналітичне рішення: 1791170049977.3193
Scikit-learn: 1791170049977.319
